In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Generate Cities List

In [15]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

642

# Perform API Calls

Perform a weather check on each city using a series of successive API calls.
Include a print log of each city as it'sbeing processed (with the city number and city name).

In [16]:
# openweathermap api url, pass it my api key (can test url by pasting in browser)
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [17]:
# Create empty lists for storing data from API
city_name = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

# Set initial count of 1 for loop
record =1

# Include a print log of each city as it'sbeing processed (with the city number and city name).
print(f"Beginning Data Retrieval")
print(f"-------------------------------")

Beginning Data Retrieval
-------------------------------


In [18]:
# fetch data for each city in list, using FOR loop (try enumerate in the FOR loop, see link from Raina)
for city in cities:
    try:        # TRY statement appends when value is not NULL (city exists in CityPy module)
        response = requests.get(query_url + city).json() 
        city_name.append(response["name"])
        cloudiness.append(response["clouds"]["all"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        humidity.append(response["main"]["humidity"])
        max_temp.append(response["main"]["temp_max"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        wind_speed.append(response["wind"]["speed"])
        city_record = response["name"]
        print(f"Processing Record {record} | {city_record}")
        print(f"{url}&q={city}")
        record= record + 1 # Increment counter  
        time.sleep(1.01) # Pause for one second in loop, to avoid maxing out API limit
    except: # EXCEPT skips to next row, where OpenWeatherMap is missing the city generated by CityPy module
        print("City not found. Skipping...")
    continue

Processing Record 1 | Nizwá
http://api.openweathermap.org/data/2.5/weather?&q=nizwa
Processing Record 2 | Dzhebariki-Khaya
http://api.openweathermap.org/data/2.5/weather?&q=dzhebariki-khaya
Processing Record 3 | Miles City
http://api.openweathermap.org/data/2.5/weather?&q=miles city
Processing Record 4 | Chai Badan
http://api.openweathermap.org/data/2.5/weather?&q=chai badan
City not found. Skipping...
Processing Record 5 | Pak Phanang
http://api.openweathermap.org/data/2.5/weather?&q=pak phanang
Processing Record 6 | Narsimhapur
http://api.openweathermap.org/data/2.5/weather?&q=narsimhapur
Processing Record 7 | Mataura
http://api.openweathermap.org/data/2.5/weather?&q=mataura
Processing Record 8 | Tongchuanshi
http://api.openweathermap.org/data/2.5/weather?&q=tongchuan
Processing Record 9 | Dingle
http://api.openweathermap.org/data/2.5/weather?&q=dingle
Processing Record 10 | Vaini
http://api.openweathermap.org/data/2.5/weather?&q=vaini
Processing Record 11 | São Filipe
http://api.ope

Processing Record 89 | Verkh-Usugli
http://api.openweathermap.org/data/2.5/weather?&q=verkh-usugli
Processing Record 90 | Kinlochleven
http://api.openweathermap.org/data/2.5/weather?&q=kinlochleven
Processing Record 91 | Port Blair
http://api.openweathermap.org/data/2.5/weather?&q=port blair
Processing Record 92 | Hithadhoo
http://api.openweathermap.org/data/2.5/weather?&q=hithadhoo
Processing Record 93 | Brae
http://api.openweathermap.org/data/2.5/weather?&q=brae
Processing Record 94 | Lata
http://api.openweathermap.org/data/2.5/weather?&q=lata
Processing Record 95 | Yellowknife
http://api.openweathermap.org/data/2.5/weather?&q=yellowknife
Processing Record 96 | Hatillo
http://api.openweathermap.org/data/2.5/weather?&q=hatillo
Processing Record 97 | East London
http://api.openweathermap.org/data/2.5/weather?&q=east london
Processing Record 98 | Chui
http://api.openweathermap.org/data/2.5/weather?&q=chuy
City not found. Skipping...
Processing Record 99 | Chokurdakh
http://api.openweath

Processing Record 176 | Quime
http://api.openweathermap.org/data/2.5/weather?&q=quime
Processing Record 177 | Sitka
http://api.openweathermap.org/data/2.5/weather?&q=sitka
Processing Record 178 | Swift Current
http://api.openweathermap.org/data/2.5/weather?&q=swift current
Processing Record 179 | Mount Gambier
http://api.openweathermap.org/data/2.5/weather?&q=mount gambier
Processing Record 180 | Varkala
http://api.openweathermap.org/data/2.5/weather?&q=varkkallai
Processing Record 181 | Hamilton
http://api.openweathermap.org/data/2.5/weather?&q=hamilton
Processing Record 182 | Celestún
http://api.openweathermap.org/data/2.5/weather?&q=celestun
Processing Record 183 | Berezovyy
http://api.openweathermap.org/data/2.5/weather?&q=berezovyy
Processing Record 184 | Bad Langensalza
http://api.openweathermap.org/data/2.5/weather?&q=bad langensalza
Processing Record 185 | Fort-Shevchenko
http://api.openweathermap.org/data/2.5/weather?&q=fort-shevchenko
Processing Record 186 | Sangar
http://api

Processing Record 261 | Ørsta
http://api.openweathermap.org/data/2.5/weather?&q=orsta
Processing Record 262 | Tabas
http://api.openweathermap.org/data/2.5/weather?&q=tabas
Processing Record 263 | Ati
http://api.openweathermap.org/data/2.5/weather?&q=ati
Processing Record 264 | Santiago del Estero
http://api.openweathermap.org/data/2.5/weather?&q=santiago del estero
Processing Record 265 | Ilhéus
http://api.openweathermap.org/data/2.5/weather?&q=ilheus
City not found. Skipping...
Processing Record 266 | Rundu
http://api.openweathermap.org/data/2.5/weather?&q=rundu
Processing Record 267 | Bambous Virieux
http://api.openweathermap.org/data/2.5/weather?&q=bambous virieux
Processing Record 268 | Cherskiy
http://api.openweathermap.org/data/2.5/weather?&q=cherskiy
Processing Record 269 | Zabul Province
http://api.openweathermap.org/data/2.5/weather?&q=zabol
Processing Record 270 | Lambaré
http://api.openweathermap.org/data/2.5/weather?&q=lambare
Processing Record 271 | Husavik
http://api.open

Processing Record 349 | Naze
http://api.openweathermap.org/data/2.5/weather?&q=naze
Processing Record 350 | Honiara
http://api.openweathermap.org/data/2.5/weather?&q=honiara
City not found. Skipping...
Processing Record 351 | Samarai
http://api.openweathermap.org/data/2.5/weather?&q=samarai
Processing Record 352 | Praia
http://api.openweathermap.org/data/2.5/weather?&q=praia
Processing Record 353 | Srednekolymsk
http://api.openweathermap.org/data/2.5/weather?&q=srednekolymsk
Processing Record 354 | Atocha
http://api.openweathermap.org/data/2.5/weather?&q=atocha
Processing Record 355 | Lavrentiya
http://api.openweathermap.org/data/2.5/weather?&q=lavrentiya
Processing Record 356 | Megion
http://api.openweathermap.org/data/2.5/weather?&q=megion
Processing Record 357 | St. George's
http://api.openweathermap.org/data/2.5/weather?&q=saint-georges
Processing Record 358 | Makakilo City
http://api.openweathermap.org/data/2.5/weather?&q=makakilo city
Processing Record 359 | Tianpeng
http://api.o

Processing Record 434 | Kawambwa
http://api.openweathermap.org/data/2.5/weather?&q=kawambwa
Processing Record 435 | Luís Correia
http://api.openweathermap.org/data/2.5/weather?&q=luis correia
City not found. Skipping...
Processing Record 436 | Nacogdoches
http://api.openweathermap.org/data/2.5/weather?&q=nacogdoches
Processing Record 437 | Eenhana
http://api.openweathermap.org/data/2.5/weather?&q=eenhana
Processing Record 438 | Līsakovsk
http://api.openweathermap.org/data/2.5/weather?&q=lisakovsk
Processing Record 439 | Kendari
http://api.openweathermap.org/data/2.5/weather?&q=kendari
Processing Record 440 | Saint Anthony
http://api.openweathermap.org/data/2.5/weather?&q=saint anthony
Processing Record 441 | Lagoa
http://api.openweathermap.org/data/2.5/weather?&q=lagoa
Processing Record 442 | Doha
http://api.openweathermap.org/data/2.5/weather?&q=doha
Processing Record 443 | Portland
http://api.openweathermap.org/data/2.5/weather?&q=portland
Processing Record 444 | Punta Alta
http://ap

Processing Record 521 | Melilla
http://api.openweathermap.org/data/2.5/weather?&q=melilla
Processing Record 522 | Kinablangan
http://api.openweathermap.org/data/2.5/weather?&q=kinablangan
Processing Record 523 | Tangping
http://api.openweathermap.org/data/2.5/weather?&q=tangping
Processing Record 524 | Arinos
http://api.openweathermap.org/data/2.5/weather?&q=arinos
Processing Record 525 | Orlik
http://api.openweathermap.org/data/2.5/weather?&q=orlik
Processing Record 526 | Chacabuco
http://api.openweathermap.org/data/2.5/weather?&q=chacabuco
Processing Record 527 | Boo
http://api.openweathermap.org/data/2.5/weather?&q=boo
Processing Record 528 | Luba
http://api.openweathermap.org/data/2.5/weather?&q=luba
Processing Record 529 | Umm Hājar
http://api.openweathermap.org/data/2.5/weather?&q=oum hadjer
Processing Record 530 | Kutum
http://api.openweathermap.org/data/2.5/weather?&q=kutum
Processing Record 531 | Campbell River
http://api.openweathermap.org/data/2.5/weather?&q=campbell river
P

# Convert Raw Data to DataFrame
Export the city data into a .csv.
Display the DataFrame

In [21]:
# Create dictionary with all the lists from API calls
weather_dictionary = {
    "City": city_name,
    "Cloudiness":cloudiness, 
    "Country":country,
    "Date":date, 
    "Humidity": humidity,
    "Lat":lat, 
    "Lng":lng, 
    "Max Temp": max_temp,
    "Wind Speed":wind_speed
}
# Create dataframe from above Dictionary
weather_df = pd.DataFrame(weather_dictionary)
weather_df
# Drop rows with humidity > 100%
weather_df.drop(weather_df[weather_df["Humidity"] > 100].index, inplace = True)
# Create and save CSV file from above DataFrame
weather_df.to_csv('output_data/cities.csv')
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nizwá,0,OM,1604282371,32,22.93,57.53,59.31,2.35
1,Dzhebariki-Khaya,0,RU,1604282372,83,62.22,135.80,-4.76,3.27
2,Miles City,1,US,1604282373,39,46.41,-105.84,50.00,5.82
3,Chai Badan,74,TH,1604282374,77,15.20,101.13,79.36,10.31
4,Pak Phanang,40,TH,1604282376,94,8.35,100.20,80.60,4.70
